In [1]:
# Import the necessary libraries
import sys
import os
import pandas as pd
import warnings

# Add the src folder to the path
sys.path.append(os.path.abspath('..'))
warnings.filterwarnings("ignore")

# Import the classes
from src.data_tool.connect_wrds import get_wrds_data, get_security_daily
from src.data_tool.yahoo_finance import yahoo_finance
from src.data_tool.ploygon import polygon
from src.data_tool.financial_datasets import FinancialDatasets
from src.data_tool.connect_db import ConnectDB
from src.data_tool.create_table import TableCreator

# Initialize the classes
yf = yahoo_finance()
pl = polygon()
fd = FinancialDatasets()

db =ConnectDB()
engine = db.get_engine()


FINANCIAL_DATASETS_API_KEY is set
YAHOO_CONSUMER_KEY and YAHOO_CONSUMER_SECRET are set
YAHOO_APP_ID is set
POLYGON_API_KEY is set
DB_USER and DB_PASSWORD are set


In [2]:
def get_database_size(db_instance, database_name):
    """
    Retrieves the total size of the database.
    """
    sql_statement = f"""
    SELECT table_schema "database_name", 
           sum(data_length + index_length) / 1024 / 1024 "database_size_mb" 
    FROM information_schema.TABLES 
    WHERE table_schema = '{database_name}'
    GROUP BY table_schema;
    """
    result = db_instance.execute_sql(sql_statement)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df

database_name = 'Elite_Traders' # Replace with your database name if different
database_size_df = get_database_size(db, database_name)
print(database_size_df)

   database_name database_size_mb
0  Elite_Traders       0.67187500


In [3]:
def get_table_sizes(db_instance):
    """
    Retrieves the size of each table in the database.
    """
    sql_statement = "SHOW TABLE STATUS"
    result = db_instance.execute_sql(sql_statement)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    size_df = df[['Name', 'Data_length', 'Index_length', 'Data_free']]
    size_df['Total_size'] = size_df['Data_length'] + size_df['Index_length']
    size_df['Data_length_MB'] = size_df['Data_length'] / 1024 / 1024
    size_df['Index_length_MB'] = size_df['Index_length'] / 1024 / 1024
    size_df['Data_free_MB'] = size_df['Data_free'] / 1024 / 1024
    size_df['Total_size_MB'] = size_df['Total_size'] / 1024 / 1024
    return size_df

table_sizes_df = get_table_sizes(db)
print(table_sizes_df)

                         Name  Data_length  Index_length  Data_free  \
0      capital_structure_debt        16384             0          0   
1    capital_structure_equity        16384             0          0   
2   capital_structure_summary        16384             0          0   
3                company_news        16384             0          0   
4           financial_metrics        16384             0          0   
5            financial_ratios        16384             0          0   
6         fundamentals_annual        16384             0          0   
7      fundamentals_quarterly        49152             0          0   
8                 identifiers        16384             0          0   
9               insider_trade        16384             0          0   
10                   position        16384             0          0   
11                      price        16384             0          0   
12             security_daily       475136             0          0   

    T

In [4]:
db.show_tables()

,Tables
0,capital_structure_debt
1,capital_structure_equity
2,capital_structure_summary
3,company_news
4,financial_metrics
5,financial_ratios
6,fundamentals_annual
7,fundamentals_quarterly
8,identifiers
9,insider_trade


In [8]:
tc = TableCreator()
tc.create_all_tables()

DB_USER and DB_PASSWORD are set
Price table already exists
Financial Metrics table already exists
Insider Trade table already exists
Company News table already exists
All tables created successfully


In [10]:
db.show_tables()

,Tables
0,capital_structure_debt
1,capital_structure_equity
2,capital_structure_summary
3,company_news
4,financial_metrics
5,financial_ratios
6,fundamentals_annual
7,fundamentals_quarterly
8,identifiers
9,insider_trade


In [11]:
db.read_table('company_news')

,id,ticker,title,author,source,date,url,sentiment
0,1,NVDA,Nvidia Stock Keeps Heading Lower. Is It Time t...,Jeremy Bowman,The Motley Fool,2025-03-31T19:37:50Z,https://www.fool.com/investing/2025/03/31/nvid...,neutral
1,2,NVDA,9 Amazing Tech Stocks Smart Investors Should N...,Jose Najarro,The Motley Fool,2025-03-31T17:34:54Z,https://www.fool.com/investing/2025/03/31/9-am...,neutral
2,3,NVDA,CoreWeave IPO Performance Likely Due To Compan...,Chris Katje,Benzinga,2025-03-31T16:12:32Z,https://www.benzinga.com/analyst-ratings/analy...,neutral
3,4,NVDA,Why Nvidia Stock Is Sinking Today,Keith Noonan,The Motley Fool,2025-03-31T16:06:54Z,https://www.fool.com/investing/2025/03/31/why-...,negative
4,5,NVDA,Nasdaq Sell-Off: After Losing Nearly $800 Bill...,The Motley Fool,The Motley Fool,2025-03-31T13:30:00Z,https://www.fool.com/investing/2025/03/31/nasd...,positive
...,...,...,...,...,...,...,...,...
263,264,NVDA,Should You Buy Nvidia Stock After Its Blowout ...,Keith Speights,The Motley Fool,2025-03-01T14:48:00Z,https://www.fool.com/investing/2025/03/01/shou...,positive
264,265,NVDA,Think AMD Stock Is Expensive? This Chart Might...,Will Healy,The Motley Fool,2025-03-01T13:45:00Z,https://www.fool.com/investing/2025/03/01/thin...,neutral
265,266,NVDA,Prediction: Broadcom Will Soar Over the Next 5...,The Motley Fool,The Motley Fool,2025-03-01T13:05:00Z,https://www.fool.com/investing/2025/03/01/pred...,neutral
266,267,NVDA,Billionaires Sell Nvidia Stock and Buy a Black...,The Motley Fool,The Motley Fool,2025-03-01T08:40:00Z,https://www.fool.com/investing/2025/03/01/bill...,neutral
